# 利用rating2inter.ipynb中U/I的index对features进行一一对应(meta-text)
- Reindex item feature ID with IDs generated in 0rating2inter.ipynb

In [25]:
import os
import pandas as pd

dataset = 'toys'
rslt_dir = f'/home/yueqi/peilin/MMRec/data/AmazonReview/{dataset}/'
item2sideFilePath = f'/home/yueqi/peilin/MMRec/AmazonRawData/Amazon_Review_Plus/{dataset}/item2side.json'
os.chdir(rslt_dir)
os.getcwd()
i_id_mapping = 'i_id_mapping.csv'
meta_data_path = rslt_dir + f'meta-{dataset}.csv'


In [26]:
# load item mapping

df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (11924, 2)


,asin,itemID
0,B001RNFQNK,1
1,158978068X,2
2,B004S8F7QM,3
3,B008ULAMSG,4


In [27]:

import gzip, json

def getMetaDF():
  metaDict = {
    'itemID': [],
    'asin': [],
    'title': [],
    'price': [],
    'imUrl': [],
    'related': [],
    'brand': [],
    'categories': [],
    'salesRank': [],
    'description': []
  }
  with open(item2sideFilePath, 'r', encoding='utf-8') as item2sideFile:
    item2side = json.load(item2sideFile)
    for itemID, sideInfo in item2side.items():
      metaDict['itemID'].append(itemID)
      metaDict['asin'].append(sideInfo.get('asin', None))
      metaDict['title'].append(sideInfo.get('title', None))
      metaDict['price'].append(sideInfo.get('price', None))
      metaDict['imUrl'].append(sideInfo.get('image', None).get('image_path', None) if sideInfo.get('image', None) is not None else None)
      metaDict['related'].append(sideInfo.get('related', None))
      metaDict['brand'].append(sideInfo.get('brand', None))
      metaDict['categories'].append(sideInfo.get('categories', None))
      metaDict['salesRank'].append(sideInfo.get('salesRank', None))
      metaDict['description'].append(sideInfo.get('description', None))
  
  return pd.DataFrame(metaDict)

meta_df = getMetaDF()

print(f'Total records: {meta_df.shape}')
meta_df[:3]

Total records: (11924, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,3536,0439893577,Little Red Tool Box: Magnetic Tabletop Learnin...,16.19,51XCjcMthML._SY300_.jpg,"{'also_bought': ['B0015KEB2M', 'B001ECI9EM', '...",Scholastic,"[Toys & Games, Dress Up & Pretend Play, Preten...",None,The Magnetic Tabletop Learning Easel is one of...
1,9066,048645195X,Dover Publications-Decorative Tile Designs Col...,1.44,61r4Gp10KTL._SY300_.jpg,"{'also_bought': ['0486456943', '0486456420', '...",Dover Pubns,"[Toys & Games, Arts & Crafts, Drawing & Painti...","{'Arts, Crafts & Sewing': 1009}",
2,5402,0545496470,The Book of Impossible Objects: 25 Eye-Popping...,11.13,61mEDfXSSYL._SY300_.jpg,"{'also_bought': ['1570548943', '1452109036', '...",Klutz,"[Toys & Games, Novelty & Gag Toys, Magic Kits ...","{'Arts, Crafts & Sewing': 1462}",


In [28]:
# dump
meta_df.to_csv(meta_data_path, index=False)
print('done!')

done!


## Reload

In [29]:
indexed_df = pd.read_csv(meta_data_path)
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

shape: (11924, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,3536,0439893577,Little Red Tool Box: Magnetic Tabletop Learnin...,16.19,51XCjcMthML._SY300_.jpg,"{'also_bought': ['B0015KEB2M', 'B001ECI9EM', '...",Scholastic,"['Toys & Games', 'Dress Up & Pretend Play', 'P...",NaN,The Magnetic Tabletop Learning Easel is one of...
1,9066,048645195X,Dover Publications-Decorative Tile Designs Col...,1.44,61r4Gp10KTL._SY300_.jpg,"{'also_bought': ['0486456943', '0486456420', '...",Dover Pubns,"['Toys & Games', 'Arts & Crafts', 'Drawing & P...","{'Arts, Crafts & Sewing': 1009}",NaN
2,5402,0545496470,The Book of Impossible Objects: 25 Eye-Popping...,11.13,61mEDfXSSYL._SY300_.jpg,"{'also_bought': ['1570548943', '1452109036', '...",Klutz,"['Toys & Games', 'Novelty & Gag Toys', 'Magic ...","{'Arts, Crafts & Sewing': 1462}",NaN
3,8316,0615444172,Original Sticker Book Starter Kit with 3 Puffy...,16.99,51ayc5M9LlL._SY300_.jpg,"{'also_bought': ['B00GOXK692', '0985090901', '...",NaN,"['Toys & Games', 'Toy Remote Control & Play Ve...",{'Toys & Games': 27977},Our Original Sticker Books follow friends and ...


In [30]:
## Reload

i_uni = indexed_df['itemID'].unique()

print(f'# of unique items: {len(i_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(i_uni), max(i_uni)))

# of unique items: 11924
min/max of unique learners: 1/11924
